In [ ]:
#specify python install location above /home/ec2-user/SageMaker so that we don't have to 
#reinstall custom packages after every reboot.

import subprocess, sys, os, site

user_libs_path="/home/ec2-user/SageMaker/.local"

if not os.path.exists(user_libs_path):
    os.makedirs(user_libs_path)

sys.path.insert(0, user_libs_path+'/lib/python3.6/site-packages')
site.USER_BASE=user_libs_path

my_env = os.environ.copy()
my_env["PYTHONUSERBASE"] = user_libs_path

process = subprocess.run("pip install -U --quiet sagemaker"
                           .split(), env=my_env, stdout=subprocess.PIPE)
process.stderr


In [ ]:
import logging
from sagemaker import Session
from time import gmtime, strftime

sagemaker_logger = logging.getLogger("sagemaker")
sagemaker_logger.setLevel(logging.INFO)
sagemaker_logger.addHandler(logging.StreamHandler())

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()


In [ ]:
from sagemaker.spark.processing import PySparkProcessor

project_bucket = 'project1-lz'
#role = 'arn:aws:iam::000378343852:role/service-role/AmazonSageMaker-ExecutionRole-20200929T192105'

spark_processor = PySparkProcessor(
    base_job_name="sm-spark",
    framework_version="2.4",
    role=role,
    instance_count=1,
    instance_type="ml.m5.xlarge",
    max_runtime_in_seconds=1200,
)

spark_processor.run(
    submit_app_py="./anonymize.py",
    arguments=['project_bucket', project_bucket,
               'input_table', 'upload',
               'output_table', 'raw',
               'database', 'default'],
    spark_event_logs_s3_uri='s3://' + bucket + '/' + prefix + '/spark_event_logs',
    logs=False
)
